# Statement

La rapidez del compresor es de 1 500 rpm. El compresor tiene un diámetro de cilindro del cilindro de 3.125 in, una carrera de 3.26 in, y
una razón biela a cigüeñal de 3.5. La presión de admisión es la atmosférica (14.7 psia), la presión pico del cilindro es de 132 psig y la presión efectiva media (mep) es de 26 psig. El flujo es de 8.9 pies cúbicos
por minuto a la presión efectiva media, lo cual da 1.6 hp.


El diámetro interior del compresor es de 3.125 in. La fuerza dinámica
que actúa sobre la cabeza varía de 0 a 1 000 lb cada ciclo, a partir
de 130 psi de presión en el cilindro. Un empaque no confinado de
asbesto-cobre de 0.06 in de espesor cubre totalmente el punto de contacto cabeza-cilindro. El espesor de la cabeza en los puntos de sujeción (exclusivas con aletas de enfriamiento) es de 0.4 in.

![](./figura_d_2.png)

In [1]:
import numpy as np
import pandas as pd
from io import StringIO
from utils import *
from sympy import Eq, solve, Symbol, symbols

In [2]:
# Asuma un perno de 5/16-18 UNC-2A
diameter_perno = 5 / 16 # in
pitch = 18 # hilos por pulgada
diameter_low = 0.2403 # in
area_tension = 0.0524 # in^2

# Compresor
D0 = 3.125 # in

# Asuma un material SAE grado 7 
SP = 105e3 # psi
SY = 115e3 # psi
SU = 133e3 # psi

# Asuma numero de pernos = 8
count_bolts = 8

# Establezca la carga maxima
P_max = 1000 # lb
P_max_unitary = P_max / count_bolts # lb

# Asuma una pre-cara de 80% de la resistencia de prueba
porcentaje_pre_carga = 0.7
pre_carga = porcentaje_pre_carga * SP * area_tension # lb
print('pre_carga =', pre_carga, 'lb')

# Modulo de young de los materiales
E_acero = 30e6 # psi
E_aluminio = 10.4e6 # psi

# Longitud de junta
l_c = 2 * diameter_perno + 1/4 # in
print('l_c =', l_c, 'in')

# Asuma que la longitud del perno es 1.25 in
l_perno = 1.25  # in
l_junta = 1.25 # in

# Calcular la longitud roscada y libre del perno / tornillo
l_s = l_perno - l_c
l_t = l_junta - l_s
print('l_s =', l_s, 'in')
print('l_t =', l_t, 'in')

# COnsideraciones 
T = 400 # F
reliability = 0.999

j = diameter_perno / l_junta # factor de junta
print('j =', j)

pre_carga = 3851.4 lb
l_c = 0.875 in
l_s = 0.375 in
l_t = 0.875 in
j = 0.25


In [3]:
table_11_8 = pd.read_csv('table_11_8.csv')
table_11_8.head()

,j,p0,p1,p2,p3
0,0.1,0.4389,-0.9197,0.8901,-0.3187
1,0.2,0.6118,-1.1715,1.0875,-0.3806
2,0.3,0.6932,-1.2426,1.1177,-0.3845
3,0.4,0.7351,-1.2612,1.1111,-0.3779
4,0.5,0.7580,-1.2632,1.0979,-0.3708


In [4]:
def get_row_values(df:pd.DataFrame, j:float):
    
    row = df.loc[df['j'] == j]
    if len(row) > 0:
        return row.iloc[0][1:].to_numpy()
    
    upper_values = df.loc[df['j'] > j].iloc[0] [1:].to_numpy()
    lower_values = df.loc[df['j'] < j].iloc[-1] [1:].to_numpy()

    upper_j = df.loc[df['j'] > j].iloc[0] [:1].to_numpy()
    lower_j = df.loc[df['j'] < j].iloc[-1] [:1].to_numpy()

    return (j - lower_j) / (upper_j - lower_j) * (upper_values - lower_values) + lower_values


In [5]:
p_0, p_1, p_2, p_3 = get_row_values(table_11_8, j)
print('p_0 =', p_0, 'p_1 =', p_1, 'p_2 =', p_2, 'p_3 =', p_3)

ratio_module = E_aluminio / E_acero
print('ratio_module =', ratio_module)

c_t = calculate_c_r(ratio_module, p_0, p_1, p_2, p_3)
print('c_t =', c_t)

p_0 = 0.6525000000000001 p_1 = -1.20705 p_2 = 1.1025999999999998 p_3 = -0.38255
ratio_module = 0.3466666666666667
c_t = 0.35062636136296305


In [6]:
area_perno = np.pi * pow(diameter_perno, 2) / 4
print('area_perno =', area_perno, 'in^2')

k_b_prima = calculate_k_b_prima(j, area_tension, area_perno, l_s=l_s, l_t=l_t)
print('k_b_prima =', f'{k_b_prima:,.4g}')

k_m = Symbol('k_m')
eqt = Eq(c_t, (k_b_prima) / (k_m + k_b_prima) )
k_m = solve(eqt)[0]
print('k_m =', f'{k_m:,.4g}')

# pRopiedades del empaque no confinado
E_asbesto_cobre = 13.5e6 # psi
espesor_empaque = 0.06 # in

area_perno = 0.07669903939428206 in^2
k_b_prima = 1.112e+06
k_m = 2.059e+6


In [7]:
# Calculate the bolt stiffness (empaque)
D_exterior = D0 + 8 * diameter_perno
print('D_exterior =', D_exterior, 'in')

area_total_empaque = np.pi * (pow(D_exterior, 2) - pow(D0, 2)) / 4
print('area_total_empaque =', area_total_empaque, 'in^2')

area_sujeta_empaque = area_total_empaque / count_bolts - area_perno # in^2
print('area_sujeta_empaque =', area_sujeta_empaque, 'in^2')

k_empaque = E_asbesto_cobre * area_sujeta_empaque / espesor_empaque
print('k_empaque =', f'{k_empaque:,.4g}')

k_mg = Symbol('k_mg')
eqt = Eq(1/k_mg, 1/k_m + 1/k_empaque)
k_mg = solve(eqt)[0]
print('k_mg =', f'{k_mg:,.4g}')

stiffness_all = k_b_prima / (k_mg + k_b_prima)
print('stiffness_all =', f'{stiffness_all:,.4g}')

D_exterior = 5.625 in
area_total_empaque = 17.18058482431918 in^2
area_sujeta_empaque = 2.0708740636456153 in^2
k_empaque = 4.659e+08
k_mg = 2.050e+6
stiffness_all = 0.3516


In [8]:
# calculate the load in the bolt and material
P_b = P_max_unitary * stiffness_all
print('P_b =', f'{P_b:,.4g}', 'lb')

P_m = (1 - stiffness_all) * P_max_unitary
print('P_m =', f'{P_m:,.4g}', 'lb') 

# Calculate the forces in the bolt and material
F_m = pre_carga - P_m
print('F_m =', f'{F_m:,.4g}', 'lb')

F_b = pre_carga + P_b
print('F_b =', f'{F_b:,.4g}', 'lb')

# Calculate the stress in the bolt and material
F_alt = (F_b - pre_carga) / 2
print('F_alt =', f'{F_alt:,.4g}', 'lb')

F_media = (F_b + pre_carga) / 2
print('F_media =', f'{F_media:,.4g}', 'lb')

k_f = calculate_k_f(diameter_perno, )
print('k_f =', f'{k_f:,.4g}')

sigma_a_nom = F_alt / area_tension
print('sigma_a_nom =', f'{sigma_a_nom:,.4g}', 'psi')

sigma_m_nom = F_media / area_tension
print('sigma_m_nom =', f'{sigma_m_nom:,.4g}', 'psi')

sigma_max = sigma_a_nom + sigma_m_nom
print('sigma_max =', f'{sigma_max:,.4g}', 'psi')

if sigma_max * k_f > SY:
    print('El perno cumple con la resistencia', sigma_max * k_f , SY)
    k_fm = (SY - k_f * sigma_a_nom) / sigma_m_nom
else:
    print('El perno NO cumple con la resistencia', sigma_max * k_f , SY)
    k_fm = k_f
print('k_fm =', f'{k_fm:,.4g}')

sigma_a = sigma_a_nom * k_f
print('sigma_a =', f'{sigma_a:,.4g}', 'psi')

sigma_m = sigma_m_nom * k_fm
print('sigma_m =', f'{sigma_m:,.4g}', 'psi')

sigma_i = pre_carga / area_tension * k_fm
print('sigma_i =', f'{sigma_i:,.4g}', 'psi')

P_b = 43.95 lb
P_m = 81.05 lb
F_m = 3,770 lb
F_b = 3,895 lb
F_alt = 21.98 lb
F_media = 3,873 lb
k_f = 5.913
sigma_a_nom = 419.4 psi
sigma_m_nom = 7.392e+4 psi
sigma_max = 7.434e+4 psi
El perno cumple con la resistencia 439556.116444175 115000.0
k_fm = 1.522
sigma_a = 2,480 psi
sigma_m = 1.125e+5 psi
sigma_i = 1.119e+5 psi


In [9]:
# Sigma t
sigma_t = pre_carga / area_tension 
print('sigma_t =', f'{sigma_t:,.4g}', 'psi')

N_y = SY / sigma_t
print('N_y =', f'{N_y:,.4g}', 'lb')

sigma_t = 7.35e+04 psi
N_y = 1.565 lb


In [10]:
N_s = (pre_carga) / (P_max_unitary * (1 - stiffness_all))
print('N_s =', f'{N_s:,.4g}', 'lb')

N_s = 47.52 lb


In [11]:
SE_prima = 0.5 * SU
print('SE_prima =', f'{SE_prima:,.4g}', 'psi')

SE_prima = 6.65e+04 psi


In [12]:
c_load = 0.7

match (diameter_perno):

    case first if diameter_perno <= 0.3:
        first_size = 0.7
        c_size = 1

    case second if diameter_perno >= 0.3 and diameter_perno <= 10:
        second_size = 0.7
        print(second, 2)
        c_size = 0.869 * pow(diameter_perno, -0.097)

    # case 

print('c_size =', c_size)

A,B = 2.7, -0.265
c_sup = A * pow(SU/1e3, B)
print('c_sup =', c_sup)

match (T):
    case T if T <= 450:
        c_temp = 1
    case T if T >= 450 and T <= 550:
        c_temp = 1 - 0.0058 * (T - 450)
    case T if T >= 550 and T <= 1020:
        c_temp = 1 - 0.0032 * (T - 840)
        
print('c_temp =', c_temp)

match reliability:
    case reliability if reliability == 0.999:
        c_reliability = 0.753
    case reliability if reliability == 0.9999:
        c_reliability = 0.702

print('c_reliability =', c_reliability)

0.3125 2
c_size = 0.9727905080521027
c_sup = 0.7388275464414432
c_temp = 1
c_reliability = 0.753


In [13]:
S_e = c_load * c_size * c_sup * c_temp * c_reliability * SE_prima
print('S_e =', f'{S_e:,.4g}', 'psi')

S_e = 2.519e+04 psi


In [16]:
print(S_e, sigma_a, sigma_m, sigma_i, sigma_t, SU)

25192.836328023557 2479.90197208733 112520.098027913 111881.676618191 73500.0 133000.0


In [15]:
N_f = (S_e * (SU - sigma_i))/ (S_e *(sigma_m - sigma_i) + SU * sigma_a)
print('N_f =', f'{N_f:,.4g}', 'lb')

N_f = 1.538 lb


In [19]:
p_prom = F_m / (np.pi*(pow(D_exterior, 2) - pow(D0, 2))/4 - count_bolts * area_tension)
print('p_prom =', f'{p_prom:,.4g}', 'psi')

N_afloje = p_prom / 130
print('N_afloje =', f'{N_afloje:,.4g}', 'lb')

p_prom = 224.9 psi
N_afloje = 1.730 lb
